In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=214dbc5d6045792253b427b3a5571ca1033b9349dd20cdbd00b1aefe78610a8c
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [4]:
from pyspark import SparkContext
sc = spark.sparkContext

In [7]:
data = sc.textFile("Capitals.txt")
data.take(10)

['Afghanistan Flag Icon \tAfghanistan \tKabul \t34,53 \t69,17\t',
 'Albania Flag Icon \tAlbania \tTirana \t41,33 \t19,82\t',
 'Algeria Flag Icon \tAlgeria \tAlgiers \t36,75 \t3,04\t',
 'American Samoa Flag Icon \tAmerican Samoa \tPago Pago \t-14,28 \t-170,70\t',
 'Andorra Flag Icon \tAndorra \tAndorra la Vella \t42,51 \t1,52\t',
 'Angola Flag Icon \tAngola \tLuanda \t-8,84 \t13,23\t',
 'Anguilla Flag Icon \tAnguilla \tThe Valley \t18,22 \t-63,06\t',
 "Antigua and Barbuda Flag Icon \tAntigua and Barbuda \tSt. John's \t17,12 \t-61,85\t",
 'Argentina Flag Icon \tArgentina \tBuenos Aires \t-34,61 \t-58,38\t',
 'Armenia Flag Icon \tArmenia \tYerevan \t40,18 \t44,51\t']

In [8]:
def new_data(line):
    city = line.split("\t")[2].replace("\t","").replace(" ","")
    c1 = float(line.split("\t")[3].replace("\t","").replace(",","."))
    c2 = float(line.split("\t")[4].replace("\t","").replace(",","."))
    return (city,c1,c2)

new_data_RDD = data.map(new_data)
new_data_RDD.take(20)

[('Kabul', 34.53, 69.17),
 ('Tirana', 41.33, 19.82),
 ('Algiers', 36.75, 3.04),
 ('PagoPago', -14.28, -170.7),
 ('AndorralaVella', 42.51, 1.52),
 ('Luanda', -8.84, 13.23),
 ('TheValley', 18.22, -63.06),
 ("St.John's", 17.12, -61.85),
 ('BuenosAires', -34.61, -58.38),
 ('Yerevan', 40.18, 44.51),
 ('Oranjestad', 12.52, -70.03),
 ('Canberra', -35.28, 149.13),
 ('Vienna', 48.21, 16.37),
 ('Baku', 40.38, 49.89),
 ('Nassau', 25.06, -77.34),
 ('Manama', 26.22, 50.58),
 ('Dhaka', 23.71, 90.41),
 ('Bridgetown', 13.1, -59.62),
 ('Minsk', 53.9, 27.57),
 ('Brussels', 50.85, 4.35)]

In [9]:
cart_rdd  = new_data_RDD.cartesian(new_data_RDD).filter(lambda x: x[0] != x[1])

cart_rdd.take(10)

[(('Kabul', 34.53, 69.17), ('Tirana', 41.33, 19.82)),
 (('Kabul', 34.53, 69.17), ('Algiers', 36.75, 3.04)),
 (('Kabul', 34.53, 69.17), ('PagoPago', -14.28, -170.7)),
 (('Kabul', 34.53, 69.17), ('AndorralaVella', 42.51, 1.52)),
 (('Kabul', 34.53, 69.17), ('Luanda', -8.84, 13.23)),
 (('Kabul', 34.53, 69.17), ('TheValley', 18.22, -63.06)),
 (('Kabul', 34.53, 69.17), ("St.John's", 17.12, -61.85)),
 (('Kabul', 34.53, 69.17), ('BuenosAires', -34.61, -58.38)),
 (('Kabul', 34.53, 69.17), ('Yerevan', 40.18, 44.51)),
 (('Kabul', 34.53, 69.17), ('Oranjestad', 12.52, -70.03))]

In [10]:
from math import radians, cos, sin, asin, sqrt

def distance(x):
    cities = str(x[0][0]) + "-" + str(x[1][0])
    lat1 = radians(x[0][1])
    lat2 = radians(x[1][1])
    lon1 = radians(x[0][2])
    lon2 = radians(x[1][2])
      
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2 
    c = 2 * asin(sqrt(a))
    r = 6371
    dist = c * r
    return(cities,dist)

In [11]:
with_distance = cart_rdd.map(distance)
with_distance.take(20)

[('Kabul-Tirana', 4334.696822428331),
 ('Kabul-Algiers', 5857.721358797605),
 ('Kabul-PagoPago', 13646.126711152365),
 ('Kabul-AndorralaVella', 5795.92545456428),
 ('Kabul-Luanda', 7601.072171051554),
 ('Kabul-TheValley', 12276.86139731658),
 ("Kabul-St.John's", 12284.795083628382),
 ('Kabul-BuenosAires', 15269.66981717785),
 ('Kabul-Yerevan', 2260.0223095298275),
 ('Kabul-Oranjestad', 13240.329603142045),
 ('Kabul-Canberra', 11356.432260346452),
 ('Kabul-Vienna', 4560.48429647794),
 ('Kabul-Baku', 1817.221977020716),
 ('Kabul-Nassau', 12506.708151931649),
 ('Kabul-Manama', 2003.9221738691908),
 ('Kabul-Dhaka', 2380.832710294535),
 ('Kabul-Bridgetown', 12451.018071564149),
 ('Kabul-Minsk', 3871.490812142648),
 ('Kabul-Brussels', 5407.780922140729),
 ('Kabul-Belmopan', 13802.713384281093)]

In [12]:
with_distance = with_distance.sortBy(lambda x: x[1],ascending=False)
with_distance.take(20)

[('Asunción-Taipei', 19927.057280169684),
 ('Taipei-Asunción', 19927.057280169684),
 ('Manama-Adamstown', 19870.180332738233),
 ('Adamstown-Manama', 19870.180332738233),
 ('Wellington-Madrid', 19854.77528376995),
 ('Madrid-Wellington', 19854.77528376995),
 ('Adamstown-Doha', 19850.398894187754),
 ('Doha-Adamstown', 19850.398894187754),
 ('Niamey-Mata-Utu', 19827.285390740144),
 ('Mata-Utu-Niamey', 19827.285390740144),
 ('Bogotá-Jakarta', 19809.53689302646),
 ('Jakarta-Bogotá', 19809.53689302646),
 ('PhnomPenh-Lima', 19796.231964289153),
 ('Lima-PhnomPenh', 19796.231964289153),
 ('Kingston-ElAaiún', 19775.642526131687),
 ('ElAaiún-Kingston', 19775.642526131687),
 ('WestIsland-SanJosé', 19748.379782136148),
 ('SanJosé-WestIsland', 19748.379782136148),
 ('Monrovia-Nukunonu', 19744.04072555748),
 ('Nukunonu-Monrovia', 19744.04072555748)]

**Asunción and Taipei are the ones that furthest	away from	each	other with 19.927 kilometers.**